In [428]:
###################################
#                                 #
#  CTU_efficiency_statistics      #
#  Akhil Garg, akhil@akhilgarg.ca #
#  Created 2021-04-22             #
#                                 #
###################################

import numpy  as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates  as mdates
import scipy.stats       as stats
import openpyxl

CTU_data   = pd.read_pickle('CTU_data.pickle')
date_frame = pd.read_pickle('Daily CTU Census.pickle')
date_frame = date_frame.sort_index().reset_index()

## Subset of data for analysis (only when CTU-G exists)

In [362]:
bolus_dates = [('2017-01-07 08:00','2017-03-15 08:00'),
               ('2018-01-07 08:00','2018-03-15 08:00')]
drip_dates  = [('2019-01-07 08:00','2019-03-15 08:00'),
               ('2020-01-07 08:00','2020-03-15 08:00')]

# Empty data frames
bolus_CTU_data   = CTU_data.iloc[0:0]
bolus_date_frame = date_frame.iloc[0:0]
drip_CTU_data    = CTU_data.iloc[0:0]
drip_date_frame  = date_frame.iloc[0:0]

for start_end in bolus_dates:
    start = pd.to_datetime(start_end[0])
    end   = pd.to_datetime(start_end[1])

    bolus_CTU_data = bolus_CTU_data.append(
          CTU_data[(CTU_data['ADMISSION_DATE_TIME'] >= start) &
                   (CTU_data['ADMISSION_DATE_TIME'] <= end)])
    
    bolus_date_frame = bolus_date_frame.append(
          date_frame[(date_frame['date'] >= start) &
                     (date_frame['date'] < end)])
    
       
for start_end in drip_dates:
    start = pd.to_datetime(start_end[0])
    end   = pd.to_datetime(start_end[1])

    drip_CTU_data = drip_CTU_data.append(
          CTU_data[(CTU_data['ADMISSION_DATE_TIME'] >= start) &
                   (CTU_data['ADMISSION_DATE_TIME'] <= end)])
    
    drip_date_frame = drip_date_frame.append(
          date_frame[(date_frame['date'] >= start) &
                     (date_frame['date'] < end)])

print('Number of admissions being analyzed')
print('Bolus: {}'.format(len(bolus_CTU_data.index)))
print('Drip:  {}'.format(len( drip_CTU_data.index)))

Number of admissions being analyzed
Bolus: 1648
Drip:  1736


### Function definitions

In [363]:
def CI_delta(x1, x2):
    '''
    Confidence intervals for a t-test
    Adapted from https://stats.stackexchange.com/a/475345
    '''
    
    n1 = x1.size
    n2 = x2.size
    m1 = np.mean(x1)
    m2 = np.mean(x2)
    
    v1 = np.var(x1, ddof=1)
    v2 = np.var(x2, ddof=1)
    
    pooled_se = np.sqrt(v1 / n1 + v2 / n2)
    delta = m1-m2
    
    tstat = delta /  pooled_se
    df = (v1 / n1 + v2 / n2)**2 / (v1**2 / (n1**2 * (n1 - 1)) + v2**2 / (n2**2 * (n2 - 1)))
    
    # two side t-test
    p = 2 * stats.t.cdf(-abs(tstat), df)
    
    # upper and lower bounds
    lb = delta - stats.t.ppf(0.975,df)*pooled_se 
    ub = delta + stats.t.ppf(0.975,df)*pooled_se
  
    return pd.DataFrame(np.array([tstat,df,p,delta,lb,ub]).reshape(1,-1),
                         columns=['T statistic','df','pvalue 2 sided','Difference in mean','lb','ub'])

In [364]:
def display_stats(bolus_series,drip_series,unit=''):
    # Describe means
    print('\nMean')
    print('Bolus: {:.2f} {}'.format(bolus_series.mean(),unit))
    print('Drip:  {:.2f} {}'.format(drip_series. mean(),unit))

    # Describe variances
    print('\nStandard deviation')
    print('Bolus: {:.2f} {}'.format(bolus_series.std(),unit))
    print('Drip:  {:.2f} {}'.format(drip_series. std(),unit))
    
    # Compare means
    print('\nMann-Whitney (Wilcoxon rank-sum) test for difference in mean:')
    print('P-value = {:.4f}'.format(stats.mannwhitneyu(
        x=bolus_series,
        y=drip_series,
        alternative='two-sided')[1]))

    # Compare variances
    print('\nLevene test for difference in variance:')
    print('P-value = {:.4f}'.format(stats.levene
        (bolus_series,drip_series)[1]))
    
    return None

## Bolus vs. drip means and variances

### ED consult time

In [365]:
print('ED consult time to admission')

# display_stats(bolus_CTU_data['consult_duration_hours'],
#               drip_CTU_data ['consult_duration_hours'],
#               unit='h')

# Describe means
print('\nMean')
print('Bolus: {:.2f} h'.format(
    bolus_CTU_data['consult_duration_hours'].mean()))
print('Drip:  {:.2f} h'.format(
    drip_CTU_data ['consult_duration_hours'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.2f} h'.format(
    bolus_CTU_data['consult_duration_hours'].std()))
print('Drip:  {:.2f} h'.format(
    drip_CTU_data ['consult_duration_hours'].std()))

# Compare means
print('\nT-test')
print('P-value = {:.4f}'.format(stats.ttest_ind(
    bolus_CTU_data['consult_duration_hours'],
    drip_CTU_data ['consult_duration_hours'])[1]))

print('\nMann-Whitney (Wilcoxon rank-sum) test for difference in mean')
print('P-value = {:.2e}'.format(stats.mannwhitneyu(
    x=bolus_CTU_data['consult_duration_hours'],
    y=drip_CTU_data ['consult_duration_hours'],
    alternative='two-sided')[1]))

# Compare variances
print('\nLevene test for difference in variance')
print('P-value = {:.4f}'.format(stats.levene
    (bolus_CTU_data['consult_duration_hours'],
     drip_CTU_data ['consult_duration_hours'])[1]))

# Compute confidence intervals
# print(welch_ttest(bolus_CTU_data['consult_duration_hours'],
#     drip_CTU_data ['consult_duration_hours']))

# Test if data is normally-distributed
# Visually it seems so per the histogram
# stats.normaltest(bolus_CTU_data['consult_duration_hours'])
# stats.normaltest(drip_CTU_data ['consult_duration_hours'])

ED consult time to admission

Mean
Bolus: 5.25 h
Drip:  4.92 h

Standard deviation
Bolus: 2.89 h
Drip:  3.00 h

T-test
P-value = 0.0009

Mann-Whitney (Wilcoxon rank-sum) test for difference in mean
P-value = 6.10e-07

Levene test for difference in variance
P-value = 0.2318


In [425]:
def admit_day_8(date):
    '''
    Given a pandas datetime object
    Returns what day the datetime object corresponds to
    Dates between midnight and 8 am count for the previous day
    '''
    
    # Admissions from 08 to 23 count for the current day
    if date.hour >= 8:
        admit_day = pd.to_datetime(
            str(admission.ADMISSION_DATE_TIME.date())+' 08:00')
        
    # Admissions after midnight (00-07) are counted for the previous day
    elif date.hour <8:
        admit_day = pd.to_datetime(
            str(admission.ADMISSION_DATE_TIME.date()-pd.Timedelta('1d'))
            +' 08:00')
        
    return admit_day

# Create a column for total daily admissions
bolus_CTU_data.loc[:,'daily_admits'] = 0
drip_CTU_data.loc [:,'daily_admits'] = 0

# Create temporary dataframes, 
# as looping over an existing dataframe while making edits
# can lead to unexpected behaviour
bolus_temp_df = bolus_CTU_data
drip_temp_df  = drip_CTU_data

# Number of admissions per day
bolus_daily_admits = bolus_date_frame.groupby('date')['admits'].sum()
drip_daily_admits  = drip_date_frame.groupby ('date')['admits'].sum()

for admission in bolus_CTU_data.itertuples():
                
    bolus_temp_df.loc[admission.Index,'daily_admits'] = \
    bolus_daily_admits.loc[admit_day_8(admission.ADMISSION_DATE_TIME)]

for admission in drip_CTU_data.itertuples():
            
    drip_temp_df.loc[admission.Index,'daily_admits'] = \
    drip_daily_admits.loc[admit_day_8(admission.ADMISSION_DATE_TIME)]
    
# Remove temporary dataframes    
bolus_CTU_data = bolus_temp_df
drip_CTU_data  = drip_temp_df
del bolus_temp_df
del drip_temp_df

bolus_CTU_data['normalized_consult_duration'] = \
    bolus_CTU_data['consult_duration_hours']/bolus_CTU_data['daily_admits']
drip_CTU_data ['normalized_consult_duration'] = \
    drip_CTU_data ['consult_duration_hours']/drip_CTU_data['daily_admits']

In [427]:
print('ED consult time to admission, normalized for number of consults')

# display_stats(bolus_CTU_data['consult_duration_hours'],
#               drip_CTU_data ['consult_duration_hours'],
#               unit='h')

# Describe means
print('\nMean')
print('Bolus: {:.2f} h/pt'.format(
    bolus_CTU_data['normalized_consult_duration'].mean()))
print('Drip:  {:.2f} h/pt'.format(
    drip_CTU_data ['normalized_consult_duration'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.2f} h/pt'.format(
    bolus_CTU_data['normalized_consult_duration'].std()))
print('Drip:  {:.2f} h/pt'.format(
    drip_CTU_data ['normalized_consult_duration'].std()))

# Compare means
print('\nT-test')
print('P-value = {:.4f}'.format(stats.ttest_ind(
    bolus_CTU_data['normalized_consult_duration'],
    drip_CTU_data ['normalized_consult_duration'])[1]))

print('\nMann-Whitney (Wilcoxon rank-sum) test for difference in mean')
print('P-value = {:.2e}'.format(stats.mannwhitneyu(
    x=bolus_CTU_data['normalized_consult_duration'],
    y=drip_CTU_data ['normalized_consult_duration'],
    alternative='two-sided')[1]))

# Compare variances
print('\nLevene test for difference in variance')
print('P-value = {:.4f}'.format(stats.levene
    (bolus_CTU_data['normalized_consult_duration'],
     drip_CTU_data ['normalized_consult_duration'])[1]))

# Compute confidence intervals
# print(welch_ttest(bolus_CTU_data['consult_duration_hours'],
#     drip_CTU_data ['consult_duration_hours']))

# Test if data is normally-distributed
# Visually it seems so per the histogram
# stats.normaltest(bolus_CTU_data['consult_duration_hours'])
# stats.normaltest(drip_CTU_data ['consult_duration_hours'])

ED consult time to admission, normalized for number of consults

Mean
Bolus: 0.42 h/pt
Drip:  0.38 h/pt

Standard deviation
Bolus: 0.28 h/pt
Drip:  0.26 h/pt

T-test
P-value = 0.0000

Mann-Whitney (Wilcoxon rank-sum) test for difference in mean
P-value = 1.36e-09

Levene test for difference in variance
P-value = 0.0032


### Variation in census size

In [314]:
# Not including CTU-E
print(bolus_date_frame[bolus_date_frame['team']!='Internal Medicine E']['census'].mean())
print(drip_date_frame [drip_date_frame ['team']!='Internal Medicine E']['census'].mean())
print(bolus_date_frame[bolus_date_frame['team']!='Internal Medicine E']['census'].std())
print(drip_date_frame [drip_date_frame ['team']!='Internal Medicine E']['census'].std())


17.439705882352943
19.17226277372263
5.57180044052262
4.08922292634864


In [315]:
print('Census size, including CTU-E\n')

# Describe means
print('Mean')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['census'].mean()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['census'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['census'].std()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['census'].std()))

# Compare means
print('\nMann-Whitney (Wilcoxon rank-sum) test for difference in mean:')
print('P-value = {:.2e}'.format(stats.mannwhitneyu(
    x=bolus_date_frame['census'],
    y=drip_date_frame ['census'],
    alternative='two-sided')[1]))

# Compare variances
print('\nLevene test for difference in variance:')
print('P-value = {:.2e}'.format(stats.levene
    (bolus_date_frame['census'],
     drip_date_frame ['census'])[1]))


##TODO
# Census size is not a weighted mean and includes CTU-E

Census size, including CTU-E

Mean
Bolus: 16.0319
Drip:  17.6496

Standard deviation
Bolus: 6.0636
Drip:  5.1621

Mann-Whitney (Wilcoxon rank-sum) test for difference in mean:
P-value = 3.22e-07

Levene test for difference in variance:
P-value = 5.70e-09


### Admission and discharge rates

In [290]:
print('Admission rate')

# Describe means
print('\nMean')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['admits'].mean()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['admits'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['admits'].std()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['admits'].std()))

# Compare means
print('\nMann-Whitney (Wilcoxon rank-sum) test for difference in mean:')
print('P-value = {:.2f}'.format(stats.mannwhitneyu(
    x=bolus_date_frame['admits'],
    y=drip_date_frame ['admits'],
    alternative='two-sided')[1]))

# Compare variances
print('\nLevene test for difference in variance:')
print('P-value = {:.2e}'.format(stats.levene
    (bolus_date_frame['admits'],
     drip_date_frame ['admits'])[1]))

print('\n')

print('Discharge rate')

# Describe means
print('\nMean')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['dischs'].mean()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['dischs'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['dischs'].std()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['dischs'].std()))

# Compare means
print('\nMann-Whitney (Wilcoxon rank-sum) test for difference in mean:')
print('P-value = {:.2f}'.format(stats.mannwhitneyu(
    x=bolus_date_frame['dischs'],
    y=drip_date_frame ['dischs'],
    alternative='two-sided')[1]))

# Compare variances
print('\nLevene test for difference in variance:')
print('P-value = {:.2f}'.format(stats.levene
    (bolus_date_frame['dischs'],
     drip_date_frame ['dischs'])[1]))

Admission rates

Mean
Bolus: 2.0466
Drip:  2.1448

Standard deviation
Bolus: 1.9803
Drip:  1.7154

Mann-Whitney (Wilcoxon rank-sum) test for difference in mean:
P-value = 0.02

Levene test for difference in variance:
P-value = 2.50e-08


Discharge rates

Mean
Bolus: 2.0380
Drip:  2.1922

Standard deviation
Bolus: 1.6399
Drip:  1.6405

Mann-Whitney (Wilcoxon rank-sum) test for difference in mean:
P-value = 0.04

Levene test for difference in variance:
P-value = 0.46


### Length of stay

In [301]:
print('Length of stay')

# Describe means
print('\nMean')
print('Bolus: {:.2f} d'.format(
    bolus_CTU_data['LOS'].mean()))
print('Drip:  {:.2f} d'.format(
    drip_CTU_data ['LOS'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.2f} d'.format(
    bolus_CTU_data['LOS'].std()))
print('Drip:  {:.2f} d'.format(
    drip_CTU_data ['LOS'].std()))

# Compare means
print('\nMann-Whitney U:')
print('P-value = {:.4f}'.format(stats.mannwhitneyu(
    bolus_CTU_data['LOS'],
    drip_CTU_data ['LOS'])[1]))

# Compare variances
print('\nLevene test for difference in variance:')
print('P-value = {:.2f}'.format(stats.levene(
     bolus_CTU_data['LOS'],
     drip_CTU_data ['LOS'])[1]))

# plt.hist(CTU_data['LOS'], bins = 14, range=(0,14))
# plt.show()

# stats.normaltest(bolus_CTU_data['LOS'])
# stats.normaltest(drip_CTU_data ['LOS'])


Length of stay

Mean
Bolus: 6.15 d
Drip:  6.37 d

Standard deviation
Bolus: 5.42 d
Drip:  6.08 d

Mann-Whitney U:
P-value = 0.2955

Levene test for difference in variance:
P-value = 0.04


### Mortality

In [305]:
print('Mortality')

# Describe means
print('\nMean')
print('Bolus: {:.2f}'.format(
    bolus_CTU_data['Death'].mean()))
print('Drip:  {:.2f}'.format(
    drip_CTU_data ['Death'].mean()))

# Describe variances
print('\nStandard deviation')
print('Bolus: {:.2f}'.format(
    bolus_CTU_data['Death'].std()))
print('Drip:  {:.2f}'.format(
    drip_CTU_data ['Death'].std()))

# Compare means
print('\nMann-Whitney U:')
print('P-value = {:.4f}'.format(stats.mannwhitneyu(
    bolus_CTU_data['Death'],
    drip_CTU_data ['Death'])[1]))

# Compare variances
print('\nLevene test for difference in variance:')
print('P-value = {:.2f}'.format(stats.levene(
     bolus_CTU_data['Death'],
     drip_CTU_data ['Death'])[1]))


#TODO
# Compare CTU_data to date_frame mortality

Mortality

Mean
Bolus: 0.07
Drip:  0.07

Standard deviation
Bolus: 0.26
Drip:  0.25

Mann-Whitney U:
P-value = 0.2135

Levene test for difference in variance:
P-value = 0.43


### Readmission rate

In [ ]:
#TODO

In [242]:
bolus_MRN_count = 0
drip_MRN_count  = 0
dfs = {'bolus':bolus_CTU_data,'drip':drip_CTU_data}
print(bolus_CTU_data['ChartNumber'].nunique())
print(drip_CTU_data['ChartNumber'].nunique())

for df in dfs.items():
    
    readmission_frame = df[1].groupby('ChartNumber',sort=False) \
    [['ADMISSION_DATE_TIME','DISCHARGE_DATE_TIME']].agg(list)

    for MRN in readmission_frame.itertuples():
    # Generates tuples of 
    #(index,admissions_timestamp_list,discharges_timestamp_list)
    
        
        # Skip MRNs that are not readmissions
        if len(MRN[1]) == 1:
            if df[0] == 'bolus': bolus_MRN_count += 1
            if df[0] == 'drip' : drip_MRN_count  += 1

        continue
        # Iterate over all admissions for a given MRN
        for i in range(len(MRN[1]) - 1):

            # Calculate admit time at second date (i+1)
            # And subtract previous discharge time (MRN[2](i))
            dc_admit_time =  MRN[1][i+1] - MRN[2][i]

            # Only keep readmissions less than 30 days
            if dc_admit_time < pd.Timedelta('30D'): 

                # Write readmision to CTU_data
                row = (CTU_data['ChartNumber']==MRN[0]) & \
                      (CTU_data['ADMISSION_DATE_TIME']==MRN[1][i+1])

                CTU_data.loc[row,'Readmission'] = True

print(bolus_MRN_count)
print(drip_MRN_count)

1257
1343
1105
1175


In [232]:
print('Mean')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['readmit_rate'].mean()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['readmit_rate'].mean()))

print('\nStandard deviation')
print('Bolus: {:.4f}'.format(
    bolus_date_frame['readmit_rate'].std()))
print('Drip:  {:.4f}'.format(
    drip_date_frame ['readmit_rate'].std()))

print('\nMean')
print('Bolus: {:.4f}'.format(
    bolus_CTU_data['Readmission'].mean()))
print('Drip:  {:.4f}'.format(
    drip_CTU_data ['Readmission'].mean()))

Mean
Bolus: 0.0187
Drip:  0.0181

Standard deviation
Bolus: 0.0393
Drip:  0.0384

Mean
Bolus: 0.1417
Drip:  0.1447


NormaltestResult(statistic=509.3868845908793, pvalue=2.443674934401823e-111)

## Write data frames to Excel

In [435]:
column_order = ['ConsultRequestDateTime',
                'ADMISSION_DATE_TIME','DISCHARGE_DATE_TIME',
                'RESIDENTSERVICE','Death','LOS','Readmission',
                'daily_admits','consult_duration_hours',
                'normalized_consult_duration']

bolus_CTU_data = bolus_CTU_data[column_order]
drip_CTU_data  = drip_CTU_data [column_order]    

with pd.ExcelWriter('Bolus_Drip_Data.xlsx') as writer:
    bolus_CTU_data.to_excel(writer,index=False,sheet_name='Bolus Admission Data')
    drip_CTU_data.to_excel (writer,index=False,sheet_name='Drip Admission Data')
    
    bolus_date_frame.to_excel(writer,index=False,sheet_name='Bolus Daily Data')
    drip_date_frame.to_excel (writer,index=False,sheet_name='Drip Daily Data')